### Assumptions Prior to Using This Module 

1. Training data has been generated via Analyzer2 (or Analyzer, a predecessor version)
   - Example input: <project_dir>/loinc_predictor/data/andromeda-pond-hepatitis-c-balanced.csv
   
   - Training data are generated in two formats 
     
     a. raw 
     
     b. encoded
   

In [1]:
import pandas as pd 
from pandas import DataFrame, Series
import os, sys, re
import numpy as np
from decimal import Decimal

import warnings
warnings.filterwarnings('ignore')  # action='once'

# local modules
from analyzer import load_data, save_data, load_performance
import loinc as lc

### Determine Target Cohort

In [2]:
cohort = 'hepatitis-c'

### Define Feature Set

In [3]:
"""
Memo
----
1. medivo_test_result_type is a function of the following attributes: 
      "meta_sender_name",
      "receiving_organization_id",
      "test_order_code",
      "test_order_name",
      "test_result_code",
      "test_result_name",
      "test_result_loinc_code",
      "test_result_units_of_measure"
      
"""
from analyzer import sample_col_values
from loinc import FeatureSet

cat_cols = FeatureSet.cat_cols
cont_cols = FeatureSet.cont_cols  # e.g. age
derived_cols = FeatureSet.derived_cols
# ... ['count']  # other possible vars: test result n-th percentile, normalized test frequency

target_cols = FeatureSet.target_cols  # ['test_result_loinc_code', ]

# cardinality < 100
low_card_cols = FeatureSet.low_card_cols # ['patient_gender', 'fasting', 'meta_sender_name' ]
high_card_cols = FeatureSet.high_card_cols

target_columns = cat_cols + cont_cols + target_cols

### Load (Curated) Training Data

note: Training data was saved prior to the variable encoding

In [ ]:
from analyzer import load_data, analyze_values

input_file=f"andromeda-pond-{cohort}-balanced.csv"
df = load_data(input_file=input_file, warn_bad_lines=False)
print("(load) dim(df): {}".format(df.shape))

# check feature values
analyze_values(df, cols=cat_cols, topn=10)  # topn: most common n feature values (and their counts)

(load_data) Loaded dataframe (dim=(1101980, 128)) from:
/Users/barnett/Documents/work/loinc_predictor/data/andromeda-pond-hepatitis-c-balanced.csv

(load) dim(df): (1101980, 128)
[1] name: patient_gender => values: 
[('F', 603447), ('M', 495872), (nan, 2661)]
 ... mode: F
[2] name: patient_state => values: 
[(nan, 141099), ('FL', 135943), ('CA', 131372), ('NY', 113619), ('GA', 87442), ('TX', 68794), ('PA', 36562), ('TN', 34938), ('NJ', 31877), ('AL', 27541)]
 ... mode: nan
[3] name: patient_bill_type => values: 
[('PRIVATE INSURANCE', 450083), ('CLIENT', 162543), ('MEDICARE', 142836), (nan, 135202), ('MANAGE CARE FFS', 89843), ('MEDICAID', 40432), ('PI', 29138), ('MANAGE CARE CAP', 17871), ('CM', 17302), ('MC', 8737)]
 ... mode: PRIVATE INSURANCE
[4] name: fasting => values: 
[(nan, 813502), ('Y', 140158), ('N', 72298), ('U', 63512), ('NOT FASTING', 8429), ('FASTING', 4081)]
 ... mode: nan
[5] name: performing_organization_id => values: 
[(nan, 399127), ('BN', 17223), ('CPLSW', 9100), 

### Feature Transformation
note: patient_date_of_birth => age

In [ ]:
from transformer import to_age
from analyzer import col_values
# from loinc import FeatureSet

tTransformed = False

if not tTransformed: 
    to_age(ts)
    values = col_values(ts, col='age', n=10)
    print("> age: {}".format(values))

# datatime columns

### Subset Features and Handling Missing Values

In [ ]:
token_default = token_missing = 'unknown'
token_other = 'other'

if not tTransformed: 

    tCategorify = False
    tDropHighMissing = False # drop columns with high rate of missing values
    p_null = 0.9

    # V = list(feature_lookup.keys())
    V = cont_cols + cat_cols + derived_cols
    L = target_cols
    dfX = df[V]
    dfy = df[L]

    print("> Given features set:\n{}\n".format(V))

    assert np.sum(dfy[target_cols[0]].isnull()) == 0

    # drop columns/vars with too many missing values 
    N = dfX.shape[0]
    n_thresh = int(N * p_null)
    nf0 = nf = dfX.shape[1]
    fset0 = set(dfX.columns.values)

    if tDropHighMissing: 
        dfX = dfX[dfX.columns[dfX.isnull().mean() < p_null]]
        fset = set(dfX.columns.values)
        nf = dfX.shape[1]
        print("> Dropped n={} features:\n{}\n".format(nf-nf0, fset0-fset))

    fset = set(dfX.columns.values)
    print("> Final feature set (nf={}):\n{}\n".format(nf, fset))

    # fill in missing values (also see default_values)
    dfX.fillna(value=token_default, inplace=True)
    #################################################
    # Convert our three categorical columns to category dtypes.

    cat_cols = [cat for cat in cat_cols if cat in dfX.columns]
    cont_cols = [c for c in cont_cols if c in dfX.columns]

### Encode Variables

In [ ]:
from transformer import encode_vars 
from loinc import FeatureSet
# high_card_cols = FeatureSet.high_card_cols

if not tTransformed: 
    nf0 = dfX.shape[1]
    dfX = encode_vars(dfX, fset=cat_cols, high_card_cols=high_card_cols)
    print("> After variable encoding we have dim(dfX): {} | nf: {} -> {}".format(dfX.shape, nf0, dfX.shape[1]))
    print("> New feature set:\n{}\n".format(dfX.columns))

### Encode Labels

In [ ]:
from analyzer import encode_labels, summarize_dict, get_sample_sizes
import collections, operator

codebook={'pos': 1, 'neg': 0, '+': 1, '-': 0}

if not tTransformed: 
    # verify
    assert dfX.shape[0] == dfy.shape[0], "> dim(dfX): {} | dfy.cols: {}".format(dfX.shape, dfy.columns.values)

    
    # choose the one with a large sample size as 'positive'
    col_label = 'test_result_loinc_code' # strings

    topn = 5
    sizes = get_sample_sizes(dfy[col_label])
    # ... sizes: (loinc) label -> sample size
    # print("> n(sizes): {}".format(len(sizes)))  # 734 for cohort='hepatitis-c'

    # Q: How many classes/codes have less than N instances? 
    N_low = 1000
    n_low = sum(1 for l, c in sizes.items() if c < N_low)
    print("> Low sample size classes | n={} (< {})".format(n_low, N_low))

    N_elow = 10
    n_elow = sum(1 for l, c in sizes.items() if c < N_elow)
    print("> Extreme low sample size classes | n={} (< {})".format(n_elow, N_elow))

    # Q: How many classes/codes were able to match the most enriched class in terms of sample size (e.g. 5707) -- by 
    #    taking in extra data from an external source? 
    eps = 10
    n_matched = sum(1 for l, c in sizes.items() if c >= max_size-eps)
    print("> n_matched: {} | max_size: {}".format(n_matched, max_size))

    # sort by values
    sizes_sorted = sorted(sizes.items(), key=operator.itemgetter(1))
    summarize_dict(sizes, topn=15, sort_=True)

    print("> sizes: {}".format(sizes.most_common(20)))
    most_sample_sizes = sizes.most_common(topn)  # take(topn, sizes.items())
    print("> Sample sizes | Top N={} codes:\n{}\n".format(topn, most_sample_sizes))
    least_sample_sizes = sizes.most_common()[:-topn-1:-1]
    print("> Sample sizss | Last N={} codes:\n{}\n".format(topn, least_sample_sizes))

    # test
    target = most_sample_sizes[0][0]
    y = encode_labels(dfy, pos_label=target, codebook=codebook, verbose=1)

### Model Training

In [ ]:
import utils_tree, utils_sys, analyzer
import collections
from analyzer import balance_by_downsampling
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from loinc import TSet

def get_sample_size(y, code_book={}):
    if not code_book: code_book = TSet.code_book
    counter = collections.Counter(y)
    return (counter[codebook['neg']], counter[codebook['pos']])

# data transformation
col = 'test_result_loinc_code'
X, y = dfX.values, dfy[col].values
print("> dim(X): {}, sample(y): {}".format(X.shape, np.random.choice(np.unique(y),20) ))

# feature scaling
scaler = MinMaxScaler() # MinMaxScaler(), StandardScaler()
X = scaler.fit_transform(X)

n_fold = 5
n_min = n_fold

# to save performance data
header = ['code', 'mean', 'std', 'n_pos']
sdict = {h:[] for h in header}
for code in loinc_set: 
    y_eff = analyzer.encode_labels(y, pos_label=code)
    
    n_neg, n_pos = get_sample_size(y_eff)
    print(f"> sample size | n0(+): {n_pos}, n0(-): {n_neg}")
    
    if n_pos >= n_min: 
        # additional step to balance control sample size (in 1-vs-all, the 'other' class tends to be too large) 
        X_eff, y_eff = balance_by_downsampling(X, y_eff, method='multiple')
        n_neg, n_pos = get_sample_size(y_eff)
        print(f"> sample size (after downsampling) | n(+): {n_pos}, n(-): {n_neg}")
        
        # training + evaluation (default classifier: logistic regression)
        scores = analyzer.eval_performance(X_eff, y_eff, model=None, cv=n_fold, random_state=53, verbose=1)
        
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        print("> average: {}, std: {}".format(mean_score, std_score))
    else: 
        print("> (positive) sample size too small, n={}".format(n_pos))
        mean_score = -1 
        std_score = -1
    sdict['code'].append(code)
    sdict['mean'].append(mean_score)
    sdict['std'].append(std_score)
    sdict['n_pos'].append(n_pos)

# --------------------------------------------------
# save performance dataframe
df_perf = DataFrame(sdict, columns=header)
df_perf = df_perf.sort_values(by=['mean', ]) # ascending=False
analyzer.save_performnace(df, output_dir='result', output_file='', **kargs)

cohort = 'hepatitis-c'
output_dir = os.path.join(os.getcwd(), 'result')
output_file = f"performance-{cohort}-3.csv" 
output_path = os.path.join(output_dir, output_file)
df_perf.to_csv(output_path, sep='|', index=False, header=True)

for code, score in zip(df_perf['code'], df_perf['mean']):
    print(f"[{code}] -> {score}")

### Visualize Results

In [ ]:
"""

Memo
---- 
1. performance plot

   perplot: https://pypi.org/project/perfplot/
"""
import seaborn as sns
import matplotlib.pyplot as plt
from analyzer import load_performance

sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 20))
sns.set_color_codes("pastel")

#---------------------------------------------

# load performance data
cohort = 'hepatitis-c'
df_perf = load_performance(input_dir='result', cohort=cohort)
print("> dim(performance matrix): {}".format(df_perf.shape))

# sort ~ performance scores 
# df_perf = df_perf.sort_values(by=['mean', ], ascending=False)

header = ['code', 'mean', 'std', 'n_pos']
codes = df_perf['code']
n_codes = len(codes)
scores = df_perf['mean']

# some statistics
score_high = 0.90
score_low = 0.50

codes_low_sz = df_perf.loc[df_perf['mean'] < 0]['code']
codes_scored = df_perf.loc[df_perf['mean'] >= 0]['code']
codes_high_score = df_perf.loc[df_perf['mean'] >= score_high]['code']
assert n_codes == len(codes_low_sz) + len(codes_scored)

print("1. Total number of codes: {} | n(low_sample): {}, n(scored):{}, n(high scored):{}".format(n_codes, 
   len(codes_low_sz), len(codes_scored), len(codes_high_score)))
r_scored = len(codes_scored)/(n_codes+0.0)
rh = len(codes_high_score)/(n_codes+0.0)
print("2. Fraction of scored codes: {}".format(r_scored))
print("3. Fraction of highly scored codes: {}".format(rh))

# Effective performance dataframe, ruling out those codes without scores (due to low sample sizes)
df_eff = df_perf.loc[df_perf['mean'] >= 0.0]

n_offset = 25
df_topn = df_eff.sort_values(['mean', ], ascending=False).head(n_offset)
df_botn = df_eff.sort_values(['mean', ], ascending=True).head(n_offset)
# print(df_botn)

# codes = [str(c) for c in df_botn['code'].values]
# print('lower codes: {}'.format(codes))
# scores = df_botn['mean'].values
# print('scores: {}'.format(scores))

# top n + bottom n
dfe = pd.concat([df_topn, df_botn], ignore_index=True)
dfe.sort_values(by=['mean', ], ascending=False, inplace=True)
codes = [str(c) for c in dfe['code'].values]
scores = dfe['mean'].values
# print('lower(n)+higher codes(n): {}'.format(codes))
# print('scores: {}'.format(scores))
print(dfe)

# sns.barplot(x="total", y="abbrev", data=crashes,
#             label="Total", color="b")

# --------------------
# ax = sns.barplot(x='mean', y='code', data=df_botn)
# print("-------------------------\n\n")
# print("> dtype: {}".format(df_botn.dtypes))
# print(df_botn.head(10))

# dfe = dfe[['mean', 'code']]
# dfe.plot(kind='bar')

sns.barplot(x='mean', y='code', data=dfe, order=dfe['code'], # order has to be specified; even if already sorted!!!
            label="LOINC", color="b", orient='h')

# ax = sns.barplot(x='mean', y='code', data=df)

# ax.set_xlabel('Fmax Score')
# ax.set_ylabel('LOINC')